In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma bs4 langchain-cohere

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
import getpass
import os

# To be able to use TogetherAI API.
os.environ["TOGETHER_API_KEY"] = getpass.getpass()

In [ ]:
from langchain_openai import ChatOpenAI

# Choose our model.
llm = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",)

In [ ]:
import getpass
import os

# Due to we'll use CohereEmbeddings.
os.environ["COHERE_API_KEY"] = getpass.getpass()

In [ ]:
from langchain_cohere import CohereEmbeddings

embeddings_model = CohereEmbeddings()

In [ ]:
# Load, chunk and index the contents of the article.
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model)

In [ ]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# Time to test our work!
rag_chain.invoke("What is Task decomposition?")

In [ ]:
# Cleanup.
vectorstore.delete_collection()